# Вебинар 6. Двухуровневые модели рекомендаций


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# !pip install lightgbm
# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from _ADDS.webinar_6.webinar_materials.metrics import precision_at_k, recall_at_k
from _ADDS.webinar_6.webinar_materials.utils import prefilter_items
from _ADDS.webinar_6.webinar_materials.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../_ADDS/webinar_6/data_init/retail_train.csv')
item_features = pd.read_csv('../_ADDS/webinar_6/data_init/product.csv')
user_features = pd.read_csv('../_ADDS/webinar_6/data_init/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


/home/arthur/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [4]:
recommender = MainRecommender(data_train_lvl_1)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [5]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1106523, 871756, 1044078, 1022428]

In [6]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [7]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 841220, 1131488, 15778319]

In [8]:
recommender.get_similar_users_recommendation(2375, N=5)

[1087411, 918046, 977559, 1096573, 1064002]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [9]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [10]:
# your_code
users_train = data_train_lvl_1['user_id'].tolist()
users_valid = result_lvl_1['user_id'].tolist()
new_users = list(set(users_valid) - set(users_train))
all_users = list(set(users_valid) & set(users_train))
result_lvl_1 = result_lvl_1[~result_lvl_1['user_id'].isin(new_users)]

recommenders = [name for name, val in MainRecommender.__dict__.items() if callable(val)][-4:]
n = 50
for r in recommenders:
    model_name_col = r.replace('get_', '').replace('_recommendations', '')
    result_lvl_1[model_name_col] = result_lvl_1['user_id'].apply(lambda x: eval(f'recommender.{r}({x}, N={n})'))
    result_lvl_1[model_name_col+'_score'] = result_lvl_1.apply(lambda x: recall_at_k(x[model_name_col], x['actual'], k=n), axis=1).mean()
    print(model_name_col, 'is ready.')

als is ready.
own is ready.
similar_items_recommendation is ready.
similar_users_recommendation is ready.


In [11]:
score_columns = [item for item in result_lvl_1.columns.tolist() if 'score' in item]
result_lvl_1[score_columns].head(1)

,als_score,own_score,similar_items_recommendation_score,similar_users_recommendation_score
0,0.052082,0.065257,0.038771,0.008119


### Задание 2.

Обучите модель 2-ого уровня, при этом:
- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
- Вырос ли precision@5 при использовании двухуровневой модели?

In [12]:
# your_code

In [13]:
# add new user_feature (средний чек)
basket_stat = user_features.merge(data, on='user_id', how='left')
basket_stat = basket_stat.pivot_table(index='user_id', values=['basket_id', 'sales_value'], aggfunc={'basket_id': 'count', 'sales_value': 'sum'})
basket_stat = basket_stat['sales_value'] / basket_stat['basket_id']
basket_stat = basket_stat.reset_index()
basket_stat.rename(columns={0: 'avg_price'}, inplace=True)
user_features = user_features.merge(basket_stat.reset_index(), on='user_id')
del basket_stat

In [14]:
# add new user_feature (популярность = количество продаж)
items_popularity = item_features.merge(data, on='item_id', how='left')
items_popularity = items_popularity.pivot_table(index='item_id', values='quantity', aggfunc='sum')
items_popularity = items_popularity.reset_index()
items_popularity.rename(columns={'quantity': 'item_pop'}, inplace=True)
item_features = item_features.merge(items_popularity, on='item_id')
del items_popularity

In [15]:
def preprocessing(data_func):
    # creating dataset for ranking
    df_match_candidates = pd.DataFrame(data_func['user_id'].unique())
    df_match_candidates.columns = ['user_id']
    df_match_candidates = df_match_candidates[df_match_candidates['user_id'].isin(data_train_lvl_1['user_id'].unique())]
    df_match_candidates['candidates'] = df_match_candidates['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=n))

    df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    df_items.name = 'item_id'
    df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

    # Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1
    df_ranker_train = data_func[['user_id', 'item_id']].copy()
    df_ranker_train['target'] = 1  # тут только покупки 
    df_ranker_train = df_match_candidates.merge(df_ranker_train, on=['user_id', 'item_id'], how='left')
    df_ranker_train['target'].fillna(0, inplace= True)

    # merging
    df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
    df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')
    
    # train split
    X = df_ranker_train.drop('target', axis=1)
    y = df_ranker_train[['target']]
    
    return [X, y]

In [16]:
X_train, y_train = preprocessing(data_train_lvl_2)

In [17]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'item_pop',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'index',
 'avg_price']

In [18]:
# ML
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

/home/arthur/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/arthur/.local/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


In [19]:
# evaluating
df_ranker_predict = X_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [20]:
df_ranker_predict.head(1)

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,item_pop,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,index,avg_price,proba_item_purchase
0,2070,1105426,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,118.0,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,657.0,2.891039,0.01142


In [21]:
X_test, y_test = preprocessing(data_val_lvl_2)

In [22]:
result_eval_ranker = X_test.groupby('user_id')['item_id'].unique().reset_index()
result_eval_ranker.columns=['user_id', 'actual']
result_eval_ranker.head(2)

# get real target answers
X_test_y = X_test.merge(y_test, right_index=True, left_index=True)
y_test_unique = X_test_y[X_test_y['target']==1.0].groupby('user_id')['item_id'].unique().reset_index()
y_test_unique.columns=['user_id', 'y_actual']
y_test_unique.head(2)

# add y_test as y_actual
result_eval_ranker = result_eval_ranker.merge(y_test_unique, on='user_id', how='left').fillna("").apply(list)

In [23]:
result_eval_ranker['own_rec'] = result_eval_ranker['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=n))

In [24]:
# evaluating by precision@5
def rerank(user_id):
    return X_test[X_test['user_id']==user_id].sort_values('predict', ascending=False).head(5).item_id.tolist()

In [25]:
# get probability predictions
X_test[cat_feats] = X_test[cat_feats].astype('category')
X_test['predict'] = lgb.predict_proba(X_test)[:, 1]
X_test = X_test.sort_values(['user_id', 'predict'], ascending=False)

In [26]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker['user_id'].apply(lambda user_id: rerank(user_id))

In [27]:
result_eval_ranker.head(1)

,user_id,actual,y_actual,own_rec,reranked_own_rec
0,1,"[856942, 9297615, 5577022, 877391, 9655212, 88...","[856942, 9297615, 5577022, 8293439, 9677939, 8...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[8293439, 8293439, 9655212, 9527558, 9297615]"


In [28]:
precision_matcher = result_eval_ranker['user_id'].apply(lambda x: precision_at_k(
                                                        result_eval_ranker.loc[result_eval_ranker['user_id']==x, 'own_rec'].squeeze(), 
                                                        result_eval_ranker.loc[result_eval_ranker['user_id']==x, 'y_actual'].squeeze(), 
                                                        k=5)).mean()
precision_ranked_matcher = result_eval_ranker['user_id'].apply(lambda x: precision_at_k(
                                                        result_eval_ranker.loc[result_eval_ranker['user_id']==x, 'reranked_own_rec'].squeeze(), 
                                                        result_eval_ranker.loc[result_eval_ranker['user_id']==x, 'y_actual'].squeeze(), 
                                                        k=5)).mean()

print(f'precision@5 of 1lvl-model is {precision_matcher}\nprecision@5 of 2lvl-model is {precision_ranked_matcher}')

/home/arthur/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


precision@5 of 1lvl-model is 0.1444117647058813
precision@5 of 2lvl-model is 0.15647058823529245


### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 